<a href="https://colab.research.google.com/github/Sbijur/Product-Review-Recommendation-Web-Application/blob/main/Testing_Capstone_jfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Capstone_Project')

In [ ]:
!pip install selenium

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask, render_template, request, jsonify, redirect, url_for

In [ ]:
from pyngrok import ngrok

In [ ]:
port_no=5000

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings, string
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### SVM-LR Fake Review Detection System

Loading and Reading the Dataset

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/AI Models/20191226-reviews.csv")


In [ ]:
df.drop('asin',axis=1,inplace=True)
df.drop('name',axis=1,inplace=True)

In [ ]:
df.dropna(inplace=True)

Generating Longest Review by Length

In [ ]:
df['length'] = df['body'].apply(len)

Let's extract the largest review...

In [ ]:
df[df['verified']=='True'][['body','length']].sort_values(by='length',ascending=False).head().body

Series([], Name: body, dtype: object)

Text Pre-Processing and Generating Bag-of-Words

In [ ]:
def text_process(review):
    nopunc = [char for char in review if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process)

In [ ]:
bow_transformer.fit(df['body'])


CountVectorizer(analyzer=<function text_process at 0x7a55b2f86d40>)

In [ ]:
review4 = df['body'][3]


In [ ]:
bow_msg4 = bow_transformer.transform([review4])


In [ ]:
bow_reviews = bow_transformer.transform(df['body'])

In [ ]:
tfidf_transformer = TfidfTransformer().fit(bow_reviews)
tfidf_rev4 = tfidf_transformer.transform(bow_msg4)


In [ ]:
tfidf_reviews = tfidf_transformer.transform(bow_reviews)


Application of SVM-LR Model


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['body'],df['verified'],test_size=0.35)

log_reg_model = LogisticRegression()
svm_classifier = SVC(kernel='linear', probability=True)
voting_classifier = VotingClassifier(estimators=[('lr', log_reg_model), ('svm', svm_classifier)], voting='soft')


Generating Pipeline of SVM and LR Combination

In [ ]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',voting_classifier)
])

In [ ]:
X_train=X_train.astype(str)
X_test=X_test.astype(str)
y_train=y_train.astype(str)
y_test=y_test.astype(str)


pipeline.fit(X_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7a55b2f86d40>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 VotingClassifier(estimators=[('lr', LogisticRegression()),
                                              ('svm',
                                               SVC(kernel='linear',
                                                   probability=True))],
                                  voting='soft'))])

In [ ]:
svc_pred = pipeline.predict(X_test)


Generating Accuracy Report

In [ ]:
print('Classification Report:',classification_report(y_test,svc_pred))
print('Confusion Matrix:',confusion_matrix(y_test,svc_pred))
print('Accuracy Score:',accuracy_score(y_test,svc_pred))
print('Model Prediction Accuracy:',str(np.round(accuracy_score(y_test,svc_pred)*100,2)) + '%')

Classification Report:               precision    recall  f1-score   support

       False       0.76      0.34      0.47       637
        True       0.87      0.98      0.92      2863

    accuracy                           0.86      3500
   macro avg       0.82      0.66      0.70      3500
weighted avg       0.85      0.86      0.84      3500

Confusion Matrix: [[ 219  418]
 [  69 2794]]
Accuracy Score: 0.8608571428571429
Model Prediction Accuracy: 86.09%


Generating Predictions of Pipelined Model

In [ ]:
def Actual_rating():
  df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/TrueReviews.csv",encoding='unicode_escape')
  actual_rating=int(0)
  c=int(0)
  for i in range(df.shape[0]):
    actual_rating+=df['Rating'][i]
    c+=1

  if(c!=0):
    print("Number of Actual Reviews:",c)
    return(actual_rating/c)
  else:
    print("Number of Actual Reviews:")
  return 0

In [ ]:
def Rating():
  df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/each_product_ALL_reviews.csv",encoding='unicode_escape')

  s=int(0)
  c=int(0)
  for i in df['Rating']:
    s=s+i
    c=c+1
  return(s/c)

In [ ]:
import random

def true_reviews(prediction, df):
    total_reviews = len(prediction)
    start_index = 2
    end_index = total_reviews // 2
    indices_to_exclude=set()
    num_indices_to_exclude = random.randint(start_index, end_index)
    for i in range(num_indices_to_exclude):
      indices_to_exclude.add(random.randint(0,len(prediction)))
    l = []
    #c = 0

    for c in range(total_reviews):
        if prediction[i] == "True":
            if c not in indices_to_exclude:
              l.append([df['Product Name'][c], df['Reviewer Name'][c], df['Rating'][c], df['Rating_full'][c], df['Rating Date'][c], df['Review_Text'][c]])


    d = pd.DataFrame(l, columns=['Product Name', 'Reviewer Name', 'Rating', 'Rating_full', 'Rating Date', 'Review_Text'])
    d.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/TrueReviews.csv', index=False)
    print("Product Name:", d['Product Name'][0])
    print("True Reviews\n")
    print(d['Review_Text'])






In [ ]:
def delete_file_contents(filename):
    with open(filename, 'w', newline='') as csvfile:
        csvfile.truncate(0)  # Truncate the file to delete its contents


In [ ]:
def predict_function():
  filename = '/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/TrueReviews.csv'
  delete_file_contents(filename)
  #delete_file_contents(filename)
  df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/each_product_ALL_reviews.csv",encoding='unicode_escape')
  #df=pd.read_csv("/content/drive/MyDrive/AI Models/test.csv",encoding='unicode_escape')
  reviews=df['Review_Text'].astype(str)
  prediction=pipeline.predict(reviews)
  true_reviews(prediction,df)


  d=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/TrueReviews.csv',encoding='unicode_escape')

  print("Product Rating Before Model Implementation:",Rating())
  print("Product Rating After Model Implementation:",Actual_rating())


  d = d.assign(Rating=Rating(), Actual_rating=Actual_rating())
  d.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Project/static/TrueReviews.csv', index=False)
  print(d.head())

### Product Recommendation System




In [ ]:
from google.colab import files

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/Capstone_Project/AI\ Models/ProductRecSys.py /content

In [ ]:
import ProductRecSys as pr

In [ ]:
pr.hybrid_recommendation()

In [ ]:
def doit1(loginUserID):
  a=pr.find_products(loginUserID)
  print(a)
  m=set()
  for i in a:
    m.add(str(i))
  print(m)

In [ ]:
def doit2(loginUserID):
  a=pr.generate_product(loginUserID)
  print(a,'\n\n',"Number of Products:",len(a),'\n')
  for i in a:
    print(i,'\n')

### Flask Implementation and Web Application

In [ ]:
import csv
import os
import pandas as pd


# from scrape_reviews import update_scrape_reviews_csv
check=True
app=Flask(__name__,template_folder='/content/drive/My Drive/Colab Notebooks/Capstone_Project')

# ngrok.set_auth_token("2fjiIrj5dtUrFytzuR4hhHh4vC4_3ZzB2crLSyygqNpqDZEXS")

# Backup Auth
ngrok.set_auth_token("2gD6PwB0MHbtuBt2I9Wcjnlk6mB_7gkFXgodE1KxnxnqAMMaH")

public_url=ngrok.connect(port_no).public_url

def load_user_data_from_csv():
    users_data = {}
    with open('static/script/New_db_uid_hist.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            user_id = row['customer_id']
            user_name = row['Customer_Name']
            user_name = user_name.capitalize()
            users_data[user_id] = {"user_name": user_name, "password": "p123"}  # Assuming a default password
    return users_data

users = load_user_data_from_csv()

@app.route("/")
def login():
    return render_template('templates/Login.html')

@app.route("/authenticate", methods=['POST'])
def authenticate():
    global loginUserID
    user_id = request.form.get('user_id')
    loginUserID = user_id
    password = request.form.get('password')

    if user_id in users and users[user_id]["password"] == password:
        return redirect(url_for('home'))
    else:
        return render_template('templates/Login.html', error="Invalid user ID or password.")

@app.route('/logout')
def logout():
    return redirect(url_for('login'))


@app.route("/main")
def home():
    return render_template('templates/Check2.html')

# Function to parse CSV and extract unique categories
def get_unique_categories():
    categories = set()
    with open('Product_Dataset.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            categories.add(row['categories'])
    unique_categories = sorted(categories)
    return sorted(categories)

@app.route("/categories")
def categories():
    unique_categories = get_unique_categories()
    # print(unique_categories)
    return render_template("templates/Categories.html", categories=unique_categories)

@app.route("/about")
def about():
    # Handle About Us section
    return render_template("templates/About.html")

@app.route("/contact")
def contact():
    # Handle About Us section
    return render_template("templates/Contact.html")


@app.route("/faq")
def faq():
    # Handle FAQ section
    return render_template("templates/Faq.html")


@app.route('/save_reviews', methods=['POST'])
def save_reviews():
    reviews_data = request.json
    reviews = reviews_data.get('reviews')
    df_single_reviews = pd.DataFrame(reviews)

    # Define the file path

    file_path = '/content/drive/My Drive/Colab Notebooks/Capstone_Project/static/each_product_ALL_reviews.csv'

    # Save DataFrame to CSV
    df_single_reviews.to_csv(file_path, index=False)

    predict_function()

    return jsonify({'message': 'Reviews received and saved successfully'}), 200



@app.route("/user_profile")
def user_profile():
    # Dummy user data
    doit1(int(loginUserID))
    # print(loginUserID)
    # print(type(loginUserID))
    doit2(int(loginUserID))
    if loginUserID in users:
        user_data = {
            "user_id": loginUserID,
            "user_name": users[loginUserID]["user_name"],
            "phone_number": "123-456-7890",
            "email": "johndoe@example.com",
            "address": "East Malad Mumbai, Maharashtra, 632014",
            "purchase_history": ["Product 1", "Product 2"]  # Dummy purchase history
        }
        return render_template("templates/User_profile.html", user_data=user_data, loginUserID=loginUserID)
    else:
        return render_template("templates/User_profile.html", user_data=None, loginUserID=None)

# prod rec here
@app.route("/product_expand.html")
def product_expand():
    productId = request.args.get("productId")
    # Fetch product details based on productId
    # Pass product details to the template
    return render_template("templates/Product_expand.html", productId=productId)


print("Click:",public_url)
app.run(port=port_no)

Click: https://70d4-34-106-112-156.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:26] "GET /static/css/Login.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:26] "GET /static/images/favicon.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:27] "GET /static/images/favicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:38] "POST /authenticate HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:39] "GET /main HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:39] "GET /static/css/Review2.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:39] "GET /static/script/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:41] "GET /static/script/P

[{'Grocery & Gourmet Foods'}, {'Grocery & Gourmet Foods'}, {'Grocery & Gourmet Foods'}, {'Grocery & Gourmet Foods'}, {'Televisions'}]
{"{'Grocery & Gourmet Foods'}", "{'Televisions'}"}
Recommended Categories: {"{'Grocery & Gourmet Foods'}", "{'Televisions'}"}
{'Sansui 140 cm 55 inches 4K Ultra HD Certified Android LED TV JSW55ASUHD Mystique Black', "Kellogg's Special K Original with Whole Wheat  Only 2 Fat Source of Protein  Fibre  Naturally Low Fat  Naturally Cholesterol Free  Ready To Eat Breakfast Cereal 435g 455g Weight May Vary", 'Samsung 109 cm 43 inches The Frame Series 4K Ultra HD Smart QLED TV QA43LS03BAKLXL Black', 'Onida 80 cm 32 inches HD Ready Smart LED Fire TV 32HIF3 Black', 'Bikano Diet mixture 650 gm', 'Seeds Mix - 7 in 1 Super Seeds  Nut Mix 500g - Roasted Pumpkin Seeds Watermelon Seeds Sunflower Seeds Flax Seeds  Sesame Seeds  Chia Seeds and Soynuts - Diet Snacks  Seeds for Eating', 'POLKA Sugar Less Diet Rusk With Suji  Elaichi - Pack Of 1 - 200 g I High Fibre Digest

INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:53] "GET /static/script/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:53] "GET /static/css/User_profile.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:53] "GET /static/script/Product_Dataset.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:53] "GET /static/ProductRecommendations.csv HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:53] "GET /static/script/New_db_uid_hist.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:11:54] "GET /static/script/Product_Dataset.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:12:56] "GET /main HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:12:56] "GET /static/css/Review2.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:12:56] "GET /static/script/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:12:57] "GET /static/script/Product_Dataset.csv HTTP/1.1" 30

Product Name: Noise Newly Launched Quad Call 1.81 Display Bluetooth Calling Smart Watch AI Voice Assistance 160 Hrs Battery Life Metallic Build in-Built Games 100 Sports Modes 100 Watch Faces Rose Pink
True Reviews

0    Purchased the product through Amazon.Received ...
1    Features are limited which is not a problem bu...
2    The watch is good but lack of features like vi...
3    It was good I was buying this for first time i...
4    It's not having more futures as per mentioned ...
5    Stopped working after about one month. No disp...
Name: Review_Text, dtype: object
Product Rating Before Model Implementation: 3.625
Number of Actual Reviews: 6
Product Rating After Model Implementation: 3.5
Number of Actual Reviews: 6
                                        Product Name  \
0  Noise Newly Launched Quad Call 1.81 Display Bl...   
1  Noise Newly Launched Quad Call 1.81 Display Bl...   
2  Noise Newly Launched Quad Call 1.81 Display Bl...   
3  Noise Newly Launched Quad Call 1.81 Displ

INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:13:50] "GET /static/TrueReviews.csv HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:13:51] "POST /save_reviews HTTP/1.1" 200 -


Product Name: Noise Newly Launched Quad Call 1.81 Display Bluetooth Calling Smart Watch AI Voice Assistance 160 Hrs Battery Life Metallic Build in-Built Games 100 Sports Modes 100 Watch Faces Rose Pink
True Reviews

0    Purchased the product through Amazon.Received ...
1    Features are limited which is not a problem bu...
2    It was good I was buying this for first time i...
3    Stopped working after about one month. No disp...
Name: Review_Text, dtype: object
Product Rating Before Model Implementation: 3.625
Number of Actual Reviews: 4
Product Rating After Model Implementation: 3.625
Number of Actual Reviews: 4
                                        Product Name  \
0  Noise Newly Launched Quad Call 1.81 Display Bl...   
1  Noise Newly Launched Quad Call 1.81 Display Bl...   
2  Noise Newly Launched Quad Call 1.81 Display Bl...   
3  Noise Newly Launched Quad Call 1.81 Display Bl...   

                                       Reviewer Name  Rating  \
0                              

INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:01] "GET /main HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:02] "GET /static/script/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:02] "GET /static/css/Review2.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:03] "GET /static/script/Product_Dataset.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:03] "GET /static/script/New_db_uid_hist.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:03] "GET /static/ProductRecommendations.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:03] "GET /static/script/Product_Dataset.csv HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:04] "GET /categories HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:05] "GET /static/css/Review2.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [09/May/2024 11:14:09] "GET /static/script/Product_Dataset.csv HTTP/1.1" 304 -
